In [1]:
import requests

In [28]:
def get_weather_nws(latitude, longitude, api_version="1.1", units="si"):
    """
    Returns:
        dict: Dictionary containing weather data or an error message on failure.
    """

    # Base URL for the NWS point forecast API
    base_url = f"https://api.weather.gov/points/{latitude},{longitude}"

    # Validate API version and units
    valid_versions = ["1.0", "1.1"]
    valid_units = ["si", "us"]
    if api_version not in valid_versions:
        raise ValueError(f"Invalid API version: {api_version}. Valid versions: {', '.join(valid_versions)}")
    if units not in valid_units:
        raise ValueError(f"Invalid units: {units}. Valid units: {', '.join(valid_units)}")

    try:
        # Get the grid ID for the given location
        response = requests.get(url=base_url)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        data = response.json()

        properties = data["properties"]

        office = properties.get("gridId", None)
        gridX = properties.get("gridX", None)
        gridY = properties.get("gridY", None)

        if not office or not gridX or not gridY:
            return {"error": "office/gridX/gridY not found in response"}


        # Build the grid forecast API URL
        # forecast_url = f"https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/{api_version}?units={units}"
        forecast_url = f"https://api.weather.gov/gridpoints/{office}/{gridX},{gridY}/forecast/"
        print("Requesting from", forecast_url)

        # Fetch the forecast using the grid ID
        forecast_response = requests.get(url=forecast_url)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()
        return forecast_data

    except requests.exceptions.RequestException as e:
        return {"error": f"Error fetching weather data: {str(e)}"}
    except ValueError as e:
        return {"error": str(e)}  # Re-raise ValueError for invalid input


In [47]:
def extract_components(weather_json):
    # print(weather_json)
    try:
        properties = weather_json["properties"]
        period = properties["periods"][1]

        temp = period['temperature'] # Fahrenheit
        rain = period['probabilityOfPrecipitation']['value'] # Probability
        humidity = period['relativeHumidity']['value'] # Percent
        wind = period['windSpeed'] # MPH

        return {"temp" : temp, "rain" : rain, "humidit" : humidity, "wind" : wind}
    except:
        return {"temp" : None, "rain" : None, "humidit" : None, "wind" : None}

In [49]:
# Usage for Champaign:
latitude = 40.11  # Champaign, Illinois (example)
longitude = -88.26

try:
    weather_json = get_weather_nws(latitude, longitude)
    if "error" in weather_json:
        print(f"Error: {weather_json['error']}")
    else:
        components = extract_components(weather_json)
        print(components)

except ValueError as e:
    print(f"Invalid input: {str(e)}")


Requesting from https://api.weather.gov/gridpoints/ILX/95,72/forecast/
{'temp': 47, 'rain': None, 'humidit': 71, 'wind': '8 mph'}
